# Repaso (Módulo 3)
> El tema principal en este módulo fue optimización. Al finalizar este módulo, se espera que ustedes tengan las siguientes competencias
> - Realizar optimizaciones de funciones escalares en un dominio dado usando sympy.
> - Dado un problema de programación lineal, llevarlo a la forma que vimos en clase y resolverlo.
> - Ajustar curvas a conjuntos de puntos dados.
> - Diseñar clasificadores binarios con regresión logística para conjuntos de datos linealmente separables.

## Ejemplo 1. Optimización de funciones escalares usando sympy

En clase vimos cómo optimizar funciones escalares dado un invervalo cerrado finito utilizando `sympy`. Ustedes, además, realizaron una tarea donde hicieron una función genérica para optimizar cualquier función dada.

Recordamos en este ejemplo como optimizar este tipo de funciones.

### 1.1 

Obtener el máximo y el mínimo absoluto de la función

$$f(x)=2x^4-16x^3+32x^2+5$$

en el intervalo $[-1, 4.5]$. Graficar la función en este intervalo, junto con el punto donde ocurre el máximo (en color rojo) y el punto donde ocurre el mínimo (en color azul).

Les recuerdo nada más como imprimir en formato LaTeX.

In [1]:
# Librería de cálculo simbólico
import sympy as sym
# Para imprimir en formato TeX
from sympy import init_printing; init_printing(use_latex='mathjax')

¿Qué más librerías necesitamos?

¿Qué más sigue?
- Declarar variable y función.

- Sacar derivada e igualar a cero.

- Evaluar en los extremos y en los puntos críticos.

- El más grande es el máximo absoluto y el más pequeño es el mínimo absoluto. Gráfico.

### 1.2
Encontrar el ángulo $\theta$ que maximiza el área de un trapecio isósceles cuya base menor y segmentos laterales miden exactamente una unidad.

Graficar la función área respecto al ángulo $\theta$ en el intervalo donde tiene sentido, junto con el punto donde ocurre el máximo (en color rojo).

- Declarar variable y función.

- Sacar derivada e igualar a cero.

- Evaluar en los extremos y en los puntos críticos.

- El más grande es el máximo absoluto.

___
## Ejemplo 2. Programación lineal

En clase vimos cómo llevar los problemas de programación lineal a la forma
\begin{equation}
\begin{array}{ll}
\min_{\boldsymbol{x}} & \boldsymbol{f}^T\boldsymbol{x} \\
\text{s. a. }        & \boldsymbol{A}_{eq}\boldsymbol{x}=\boldsymbol{b}_{eq} \\
                     & \boldsymbol{A}\boldsymbol{x}\leq\boldsymbol{b}.
\end{array}
\end{equation}

Además, aprendimos a resolver los problemas en esta forma con la función `linprog` del paquete `pyomo_utilities.py`, proporcionando únicamente los parámetros $\boldsymbol{f}$, $\boldsymbol{A}$ y $\boldsymbol{b}$ ($\boldsymbol{A}_{eq}$ y $\boldsymbol{b}_{eq}$, de ser necesario). 

### 2.1
Maximizar la función $x_1+2x_2+3x_3+4x_4+5$ sujeta a las restricciones $4x_1+3x_2+2x_3+x_4\leq10$, $x_1−x_3+2x_4=2$, $x_1+x_2+x_3+x_4\geq1$, $x_1\geq0$, $x_2\geq0$, $x_3\geq0$, $x_4\geq0$.

### 2.2

Una dieta ideal debe satisfacer (o posiblemente, exceder) ciertos requerimientos nutricionales básicos al menor costo posible, ser variada y buena al paladar. ¿Cómo podemos formular dicha dieta?

Suponga que solo tenemos acceso a las siguientes comidas:

In [35]:
import pandas as pd

In [36]:
df = pd.DataFrame(columns=['Energia', 'Proteina', 'Calcio', 'Precio', 'Limite_diario'], index=['Avena', 'Pollo', 'Huevos', 'Leche', 'Pastel', 'Frijoles_cerdo'])
df.loc[:,'Energia']=[110, 205, 160, 160, 420, 260]
df.loc[:,'Proteina']=[4, 32, 13, 8, 4, 14]
df.loc[:,'Calcio']=[2, 12, 54, 285, 22, 80]
df.loc[:,'Precio']=[3, 24, 13, 9, 20, 19]
df.loc[:,'Limite_diario']=[4, 3, 2, 8, 2, 2]

df

,Energia,Proteina,Calcio,Precio,Limite_diario
Avena,110,4,2,3,4
Pollo,205,32,12,24,3
Huevos,160,13,54,13,2
Leche,160,8,285,9,8
Pastel,420,4,22,20,2
Frijoles_cerdo,260,14,80,19,2


Luego de consultar expertos en nutrición tenemos que una dieta satisfactoria tiene por lo menos $2000$ kcal de energía, $55$ g de proteina, y $800$ mg de calcio.

Para imponer la variedad se ha decidido limitar el número de porciones diarias de cada una de las comidas como se indica en la tabla.

<script>
  $(document).ready(function(){
    $('div.prompt').hide();
    $('div.back-to-top').hide();
    $('nav#menubar').hide();
    $('.breadcrumb').hide();
    $('.hidden-print').hide();
  });
</script>

<footer id="attribution" style="float:right; color:#808080; background:#fff;">
Created with Jupyter by Esteban Jiménez Rodríguez.
</footer>